# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-12-23 17:17:56] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-12-23 17:17:56] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-12-23 17:17:56] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-23 17:17:58] WARNING server_args.py:1529: Attention backend not specified. Use fa3 backend by default.


[2025-12-23 17:17:58] INFO server_args.py:2405: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=104 avail_mem=76.29 GB):  15%|█▌        | 3/20 [00:00<00:01,  8.92it/s]

Capturing batches (bs=72 avail_mem=76.27 GB):  25%|██▌       | 5/20 [00:01<00:03,  4.69it/s]

Capturing batches (bs=24 avail_mem=76.25 GB):  55%|█████▌    | 11/20 [00:01<00:00, 10.91it/s]

Capturing batches (bs=4 avail_mem=76.22 GB):  80%|████████  | 16/20 [00:01<00:00, 14.31it/s]

Capturing batches (bs=1 avail_mem=76.22 GB): 100%|██████████| 20/20 [00:01<00:00, 11.47it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Patsy and I am a research scientist and expert in oceanography. I'm a PhD student at the University of Alaska's Anchorage campus, and I have a keen interest in understanding the causes and consequences of sea level rise and its impacts on coastal communities. I have worked at the U.S. Geological Survey in Alaska and at the University of Alaska, and I've researched issues related to sea level rise, coastal erosion, and coastal flooding on a national and international level. My research has included writing and reviewing books, and I have presented my research at the national and international levels. My research has also been recognized in the peer-reviewed
Prompt: The president of the United States is
Generated text:  trying to estimate how many jellybeans are in a large jar of jellybeans. He randomly samples 100 jellybeans and finds that 75 of them are red. He wants to estimate the total number of jellybeans in the jar with a 95% confidence l

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is the largest city in Europe and the third-largest city in the world by population. It is known for its rich history, beautiful architecture, and vibrant culture. Paris is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city is also known for its fashion industry and has produced many famous artists, writers, and musicians. Paris is a popular tourist destination and a major economic center in Europe. It is a cultural and intellectual hub that attracts millions of visitors each year. The city is also home to many important institutions such as the French Academy of Sciences and

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends:

1. Increased automation and robotics: As AI continues to advance, we can expect to see more automation and robotics in various industries, from manufacturing to healthcare. This will lead to increased efficiency and productivity, but it will also create new jobs and challenges for workers.

2. AI-powered healthcare: AI will play a key role in improving the accuracy and efficiency of healthcare delivery. This will include the use of AI to analyze medical data, predict patient outcomes, and develop personalized treatment plans.





### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name] and I'm a [Your Profession] with [Your Industry] experience. I have a passion for [Your Field of Interest], and I'm always up for learning new things and trying new things. I'm excited to learn more about you and how I can help you achieve your goals. Let's get to know each other better. [Your Name] (or any other initials) [Your Age] [Your Job Title] [Your Industry] [Your Field of Interest] [Your Motivation] [Your Strengths] [Your Weaknesses] [Your Interests] [Your Personality] [Your Goals

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

Explain the following sentences. 
1. The capital of France is Paris. 
2. The capital of France is Paris. 
3. The capital of France is Paris. 

Provide a concise factual statement about France’s capital city. The capital city of France is Paris. 

Explain t

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 as

 voice

 assistants

 or

 autonomous

In [6]:
llm.shutdown()